Data Mining - MSDS 7331 - Thurs 6:30, Summer 2016
Team 3 (AKA Team Super Awesome):  Sal Melendez, Rahn Lieberman, Thomas Rogers


Github page:
https://github.com/RahnL/DataScience-SMU/tree/master/DataMining

<hr>

# Data Mining - Mini Lab 1


- Import the data

In [3]:
import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.read_csv("data/LLCP2014XPT.txt", sep="\t")
df.head()

C:\Users\rahnl\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_FOBTFS,_CRCREC,_AIDTST3,_IMPEDUC,_IMPMRTL,_IMPHOME,RCSBRAC1,RCSRACE1,RCHISLA1,RCSBIRTH
0,1,1,1172014,1,17,2014,1100,2014000001,2014000001,1.0,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN
1,1,1,1072014,1,7,2014,1100,2014000002,2014000002,1.0,...,2.0,2.0,2.0,4,1,1,NaN,NaN,NaN,NaN
2,1,1,1092014,1,9,2014,1100,2014000003,2014000003,1.0,...,2.0,2.0,2.0,6,1,1,NaN,NaN,NaN,NaN
3,1,1,1072014,1,7,2014,1100,2014000004,2014000004,1.0,...,2.0,1.0,2.0,6,3,1,NaN,NaN,NaN,NaN
4,1,1,1162014,1,16,2014,1100,2014000005,2014000005,1.0,...,2.0,1.0,2.0,5,1,1,NaN,NaN,NaN,NaN


To start, lets try to predict Age based on general Health (GENHLTH), income and whether they are a smoker.
 
 We will do some cleanup first:

In [23]:
df_reduced = df[['_AGE80','GENHLTH','_INCOMG', '_SMOKER3']]
df_reduced.head()

# smoker==3 (former smoker) and smoker==4 (never smoked) and smoker==9 (no answer) can be combined to smoker=0.
# Is this OK?

,_AGE80,GENHLTH,_INCOMG,_SMOKER3
0,61,5.0,5,3
1,73,3.0,2,4
2,52,1.0,5,3
3,67,3.0,3,4
4,67,3.0,2,4


## Training And Testing

Fivefold cross validation, using 80% of data as training.

In [32]:
from sklearn.cross_validation import ShuffleSplit

#... setup x, y
#if '_Age80' in df_reduced:
y = df_reduced['_AGE80'].values # get the labels we want
del df_reduced['_AGE80'] # get rid of the class label
X = df_reduced.values # use everything else to predict!

# do the cross validation
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
print cv_object

ShuffleSplit(464664, n_iter=3, test_size=0.2, random_state=None)


In [33]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object: 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print "====Iteration",iter_num," ===="
    print "accuracy", acc 
    print "confusion matrix\n",conf
    iter_num+=1
    
# Also note that every time you run the above code
#   it randomly creates a new training and testing set, 
#   so accuracy will be different each time

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').